# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

Ensure custom resource validation is activated

In [1]:
!minikube start --feature-gates=CustomResourceValidation=true

Starting local Kubernetes v1.8.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


Install Helm

In [2]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


Label the node to allow load testing to run on it

In [3]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "validation" not labeled


## Set up REST and gRPC methods

In [34]:
import requests
from requests.auth import HTTPBasicAuth
from grpc_python import prediction_pb2
from grpc_python import prediction_pb2_grpc
import grpc

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://192.168.99.100:30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://192.168.99.100:30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("192.168.99.100:30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [43]:
!helm install ../../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set cluster_manager.image.tag=0.3-SNAPSHOT \
    --set apife.image.tag=0.1-SNAPSHOT \
    --set engine.image.tag=0.2-SNAPSHOT

NAME:   seldon-core
LAST DEPLOYED: Mon Dec 18 08:34:46 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     2s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           2s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          2s
seldon-apiserver         1        1        1           0          2s
seldon-cluster-manager   1        1        1           0          2s
grafana-prom-deployment  1        1        1           0          2s
kafka                    1        1        1           0          2s
prometheus-deployment    1        1        1           0          2s
redis                    1        1        1           0          2s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)          

In [5]:
!helm status seldon-core

LAST DEPLOYED: Sun Dec 17 15:29:28 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           10s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          10s
seldon-apiserver         1        1        1           0          10s
seldon-cluster-manager   1        1        1           0          10s
grafana-prom-deployment  1        1        1           0          10s
kafka                    1        1        1           0          10s
prometheus-deployment    1        1        1           0          10s
redis                    1        1        1           0          10s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>       

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [6]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha1", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"clusterName":"", "deletionGracePeriodSeconds":(*int64)(nil), "uid":"2b4ff8b8-e33f-11e7-b7b1-080027a3a01a", "selfLink":"", "deletionTimestamp":interface {}(nil), "creationTimestamp":"2017-12-17T15:30:10Z", "initializers":interface {}(nil), "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "namespace":"default"}, "spec":map[string]interface {}{"oauth_key":1234, "oauth_secret":"oauth-secret", "predictors":[]interface {}{map[string]interface {}{"name":"fx-market-predictor", "replicas":1, "annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [7]:
!kubectl create -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" created


In [8]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[description:Can't find container for predictive unit with name mean-classifier_ state:FAILED]

In [9]:
!kubectl delete -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" deleted


## Normal Operation

In [10]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
       

## Create Seldon Deployment

In [11]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [12]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   <invalid>


In [13]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2017-12-17T15:30:53Z
  Generation:          0
  Initializers:        <nil>
  Resource Version:    487468
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 449c3b58-e33f-11e7-b7b1-080027a3a01a
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Name:                    test-deployment
  Oauth _ Key:             oauth-key
  Oauth _ Secret:          oauth-secret
  Predictors:
    Annotations:
      Predictor 

In [15]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [40]:
rest_request()

{
  "meta": {
    "puid": "9bhp0nfqt9gjo0baso3a3c8gkd",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [41]:
grpc_request()

meta {
  puid: "ar4u7iprikbe3109gilooceqql"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Update deployment with canary

In [35]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

In [92]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


In [94]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1] map[name:test-deployment-fx-market-predictor-canary replicas:1 replicasAvailable:1]]]

#### REST Request

In [38]:
rest_request()

{
  "meta": {
    "puid": "kvlc3jp1j4866oiifngvo7ihfh",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC request

In [39]:
grpc_request()

meta {
  puid: "501k5sflgqs7bhi1un25qdmumj"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Load test

In [98]:
!helm install ../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret 

NAME:   loadtest
LAST DEPLOYED: Wed Dec  6 20:51:59 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP     EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.98.156.223  <none>       5557:31903/TCP,5558:31975/TCP,8089:31839/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-pg47v   0/1    ContainerCreating  0         0s
locust-master-1-dxrcp  0/1    ContainerCreating  0         0s
locust-master-1-pzhzj  0/1    Terminating        0         55m




# Tear down

In [99]:
!helm delete loadtest --purge

release "loadtest" deleted


In [100]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [42]:
!helm delete seldon-core --purge

release "seldon-core" deleted
